In [1]:
# if the package is not installed in your python environment, run this to execute the notebook directly from inside the GitHub repository
%cd -q ..

In [97]:
import ipywidgets as widgets
from ipywidgets import Checkbox, VBox, HBox, Label, Text, AppLayout, Button, Layout
from IPython.display import clear_output

import weldx
from weldx.constants import WELDX_QUANTITY as Q_
from weldx.asdf.tags.weldx.core.iso_groove import get_groove
import matplotlib.pyplot as plt

In [126]:
plot_layout = Layout(width="60%", height="350px")
button_layout = Layout(width="200px", height="50px")
textbox_layout = Layout(width="15%", height="25px")
description_layout = Layout(width="30%", height="25px")

In [127]:
def hbox_FloatText_creator(text, unit, value=7.5):
    hbox = HBox([
        Label(text + " :", layout=description_layout),
        widgets.BoundedFloatText(value=value, min=0, layout=textbox_layout),
        # TODO change label to text window to select own mesurement! with default...
        Text(value=unit, placeholder='unit', layout=textbox_layout),
    ])
    return hbox

In [130]:
# all attributes used in get_groove - can we import them somehow??!
attr_list = ['workpiece_thickness',
             'workpiece_thickness2',
             'root_gap',
             'root_face',
             'root_face2',
             'root_face3',
             'bevel_radius',
             'bevel_radius2',
             'bevel_angle',
             'bevel_angle2',
             'groove_angle',
             'groove_angle2',
             'special_depth',
             'code_number']

# create dict with hboxes of all attributes
hbox_dict = dict()
for item in attr_list:
    if item == "code_number":
        d = widgets.Dropdown(options=["1.12", "1.13", "2.12", "3.1.1", "3.1.2", "3.1.3", "4.1.1", "4.1.2", "4.1.3"], layout=description_layout)
        hbox_dict[item] = HBox([Label("code_number :", layout=description_layout), d])
    elif "angle" in item:
        hbox_dict[item] = hbox_FloatText_creator(item, "deg", 45)
    elif "workpiece_thickness" in item:
        hbox_dict[item] = hbox_FloatText_creator(item, "mm", 15)
    else:
        hbox_dict[item] = hbox_FloatText_creator(item, "mm", 5)

groovetype_dropdown = widgets.Dropdown(
    # can we import those options form anywhere?!?
    options=["None",
             "VGroove",
             "UGroove",
             "IGroove",
             "UVGroove",
             "VVGroove",
             "HVGroove",
             "HUGroove",
             "DoubleVGroove",
             "DoubleUGroove",
             "DoubleHVGroove",
             "DoubleHUGroove",
             "FrontalFaceGroove"],
    value='None',
    description='Groove type :',
)

selected_values_vbox = VBox([])

# dict with groves Maybe import from anywhere?!?
groove_dict = dict()
groove_dict["None"] = []
groove_dict["VGroove"] = ["workpiece_thickness", "groove_angle", "root_gap", "root_face"]
groove_dict["UGroove"] = ["workpiece_thickness", "bevel_angle", "bevel_radius", "root_gap", "root_face"]
groove_dict["IGroove"] = ["workpiece_thickness","root_gap"]
groove_dict["UVGroove"] = ["workpiece_thickness", "groove_angle", "bevel_angle", "bevel_radius", "root_gap", "root_face"]
groove_dict["VVGroove"] = ["workpiece_thickness", "groove_angle", "bevel_angle", "root_gap", "root_face", "root_face2"]
groove_dict["HVGroove"] = ["workpiece_thickness", "bevel_angle", "root_gap", "root_face"]
groove_dict["HUGroove"] = ["workpiece_thickness", "bevel_angle", "bevel_radius", "root_gap", "root_face"]
groove_dict["DoubleVGroove"] = ["workpiece_thickness", "groove_angle", "groove_angle2", "root_gap", "root_face", "root_face2", "root_face3"]
groove_dict["DoubleUGroove"] = ["workpiece_thickness", "bevel_angle", "bevel_angle2", "bevel_radius", "bevel_radius2", "root_gap", "root_face", "root_face2", "root_face3"]
groove_dict["DoubleHVGroove"] = ["workpiece_thickness", "bevel_angle", "bevel_angle2", "root_gap", "root_face", "root_face2", "root_face3"]
groove_dict["DoubleHUGroove"] = ["workpiece_thickness", "bevel_angle", "bevel_angle2", "bevel_radius", "bevel_radius2", "root_gap", "root_face", "root_face2", "root_face3"]
groove_dict["FrontalFaceGroove"] = ["workpiece_thickness", "workpiece_thickness2", "groove_angle", "root_gap", "root_face", "special_depth", "code_number"]

o = widgets.Output(layout=plot_layout)
actual_groove = None
        
def update_plot(change):
    if groovetype_dropdown.value != "None":
        with o:
            clear_output()
            get_groove_dict = dict()
            get_groove_dict["groove_type"] = groovetype_dropdown.value
            for child in selected_values_vbox.children:
                if child.children[0].value[:-2] == "code_number":
                    get_groove_dict[child.children[0].value[:-2]] = child.children[1].value
                else:
                    get_groove_dict[child.children[0].value[:-2]] = Q_(child.children[1].value, child.children[2].value)

            actual_groove = get_groove(**get_groove_dict)
            actual_groove.plot()
            plt.show()
    else:
        with o:
            clear_output()

def remove_not_selected(change):
    if groovetype_dropdown.value == "None":
        selected_values_vbox.children = []
    else:
        # select correct sliders
        selected_values_vbox.children = [slider for key, slider in hbox_dict.items() if key in groove_dict[groovetype_dropdown.value]] 

button_o = widgets.Output()        

def on_button_clicked(b):
    with button_o:
        clear_output()
        buffer = weldx.asdf.utils._write_buffer(actual_groove)
        weldx.asdf.utils.notebook_fileprinter(buffer)
        
# observe
groovetype_dropdown.observe(remove_not_selected, names='value')
groovetype_dropdown.observe(update_plot, "value")
for _, box in hbox_dict.items():
    box.children[1].observe(update_plot, "value")


# button
save_as_asdf = Button(
    description= "Show as .yml File",
    layout=button_layout
)
save_as_asdf.on_click(on_button_clicked)

# create rest
update_plot(None)
groove_selection = VBox([groovetype_dropdown, selected_values_vbox])
result = VBox([HBox([groove_selection, o]), save_as_asdf, button_o])
 
display(result)




# TODO: code_number should change the choosen attributes or disable those not needed?!
# TODO: add layout - even out the boxes...
# TODO: asdf save button